In [2]:
import pandas as pd
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import streamlit as st

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


import nltk
nltk.download('punkt')  # Necesario para la tokenización
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords # stopwords, palabras vacías
nltk.download('stopwords')

import spacy # lematización

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package punkt to /Users/caterina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/caterina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/caterina/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
df = pd.read_csv("Dataset/df_analisis_general1.csv")

In [4]:
df["Title"]= df["Title"].str.title()

In [5]:
# me quedo con las sieries que tienen reseñas
df = df[df["Reviews"]!= "No reviews"]

In [6]:
df.info() # tengo 829 series

<class 'pandas.core.frame.DataFrame'>
Index: 829 entries, 8 to 9605
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          829 non-null    int64  
 1   Title               829 non-null    object 
 2   Synopsis            827 non-null    object 
 3   Genero              825 non-null    object 
 4   Rating              829 non-null    float64
 5   Popularity          829 non-null    int64  
 6   Vote Count          829 non-null    float64
 7   Original Language   829 non-null    object 
 8   Origin Country      829 non-null    object 
 9   First Air Date      829 non-null    object 
 10  In production       829 non-null    bool   
 11  Status              829 non-null    object 
 12  Number of Season    829 non-null    int64  
 13  Number of Episodes  829 non-null    int64  
 14  Episode Duration    771 non-null    float64
 15  Where to Watch      829 non-null    object 
 16  Reviews     

In [7]:
df["Reviews"].value_counts()

Reviews
American idol, a season and series juggernaut, one of the most incredible and unique competitions to ever spark the world with the possibility of an unknown person becoming a mega star.\r\n\r\nRandy, Paula and Simon, iconic names on their own, but even more when you combine them together. Randy Jackson was a music mogul, an individual who played for Mariah Carey, and a legend in his own right. Paula Abdul music icon from the 80s, one of the biggest selling artist and amazing choreographers ever to step foot on a stage. And then Simon Cowell, the ruthless, aggressive and honest individual that America fell in love with. And has went on to become one of the Biggest Music executive directors in the world.\r\n\r\nI’m going to captivate all seasons in this one review, so this is going to be a long one, so enjoy. \r\n\r\nSeason one, what an incredible journey, a show that you did not think was going to last let alone become such a mega hit that it became. But more importantly, you ha

In [8]:
def split_reviews(df):
    # Separar las reseñas en listas utilizando el delimitador ' | '
    df["Reviews"] = df["Reviews"].str.split(" \| ")
    
    # Usar explode() para separar las listas en filas individuales
    df_expanded = df.explode("Reviews")
    
    return df_expanded

# Aplicar la función para separar las reseñas
df_reviews_separated = split_reviews(df)

<>:3: SyntaxWarning: invalid escape sequence '\|'
<>:3: SyntaxWarning: invalid escape sequence '\|'
/var/folders/2x/t9dxtxdd28d5d8ddpkgy0y200000gq/T/ipykernel_80394/3797113215.py:3: SyntaxWarning: invalid escape sequence '\|'
  df["Reviews"] = df["Reviews"].str.split(" \| ")


In [9]:
df_reviews_separated

,Unnamed: 0,Title,Synopsis,Genero,Rating,Popularity,Vote Count,Original Language,Origin Country,First Air Date,...,Status,Number of Season,Number of Episodes,Episode Duration,Where to Watch,Reviews,Cast,Poster,Trailer,release_year
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,Returning Series,3,22,60.0,"Netflix basic with Ads, Netflix",Just totally worth it.,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,Returning Series,3,22,60.0,"Netflix basic with Ads, Netflix","Visually excellent, the dark broody streets of...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,Returning Series,3,22,60.0,"Netflix basic with Ads, Netflix","A perfect opening and great second episode , i...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,Returning Series,3,22,60.0,"Netflix basic with Ads, Netflix",extremely disappointing idea theft\r\nSo once ...,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,Returning Series,3,22,60.0,"Netflix basic with Ads, Netflix","Whilst not an entirely original idea, ""Squid G...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,9657,The Last Man On Earth,"The year is 2022, and after an unlikely event,...","Drama, Comedy",6.900,44,865.0,English,United States,2015-03-01,...,Canceled,4,67,22.0,"Hulu, Sky Go, Tele2 Play, Disney Plus, TV 2 Play",Contains spoilers of the first season.\r\n\r\n...,"Will Forte, Kristen Schaal, Cleopatra Coleman,...",https://image.tmdb.org/t/p/w500/9TcvdOIBEnIDXb...,No trailer available,2015
9582,9658,So I Married An Anti-Fan,The romance between top star Hoo Joon and his ...,"Comedy, Drama",7.420,44,50.0,Korean,"Korea, Republic of",2021-04-30,...,Ended,1,16,60.0,"Amazon Prime Video with Ads, wavve, Hulu, Raku...","_""So I married my Anti-fan""_ was a light and f...","Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",https://image.tmdb.org/t/p/w500/nfyJOt9HjdoF9q...,No trailer available,2021
9582,9658,So I Married An Anti-Fan,The romance between top star Hoo Joon and his ...,"Comedy, Drama",7.420,44,50.0,Korean,"Korea, Republic of",2021-04-30,...,Ended,1,16,60.0,"Amazon Prime Video with Ads, wavve, Hulu, Raku...",Within less than 3 minutes going into the firs...,"Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",https://image.tmdb.org/t/p/w500/nfyJOt9HjdoF9q...,No trailer available,2021
9583,9659,Porridge,Porridge is a British situation comedy broadca...,Comedy,8.127,44,51.0,English,United Kingdom,1974-09-05,...,Ended,3,18,30.0,"Foxtel Now, BritBox Amazon Channel, Sky Go",It's hard to believe that there were only ever...,"Ronnie Barker, Richard Beckinsale, Fulton Mack...",https://image.tmdb.org/t/p/w500/qSoHitLrJLNALU...,No trailer availab

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
# create preprocess_text function
def preprocess_text(text):
    # tekenizar
    tokens = word_tokenize(text.lower())

    # eliminar stopwords
    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]

    # lematizar
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos = "v")for token in filtered_tokens]

    return " ".join(lemmatized_tokens)

In [12]:
df_reviews_separated["Lemmatized_reviews"] = df_reviews_separated["Reviews"].apply(preprocess_text)
df_reviews_separated

,Unnamed: 0,Title,Synopsis,Genero,Rating,Popularity,Vote Count,Original Language,Origin Country,First Air Date,...,Number of Season,Number of Episodes,Episode Duration,Where to Watch,Reviews,Cast,Poster,Trailer,release_year,Lemmatized_reviews
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,3,22,60.0,"Netflix basic with Ads, Netflix",Just totally worth it.,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021,totally worth .
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,3,22,60.0,"Netflix basic with Ads, Netflix","Visually excellent, the dark broody streets of...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021,"visually excellent , dark broody streets run n..."
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,3,22,60.0,"Netflix basic with Ads, Netflix","A perfect opening and great second episode , i...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021,"perfect open great second episode , let averag..."
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,3,22,60.0,"Netflix basic with Ads, Netflix",extremely disappointing idea theft\r\nSo once ...,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021,extremely disappoint idea theft `` dangerous g...
8,8,Squid Game,Hundreds of cash-strapped players accept a str...,"Action & Adventure, Mystery, Drama",7.900,1938,15382.0,Korean,"Korea, Republic of",2021-09-17,...,3,22,60.0,"Netflix basic with Ads, Netflix","Whilst not an entirely original idea, ""Squid G...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",https://image.tmdb.org/t/p/w500/1QdXdRYfktUSON...,https://www.youtube.com/watch?v=oqxAJKy0ii4,2021,"whilst entirely original idea , `` squid game ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9581,9657,The Last Man On Earth,"The year is 2022, and after an unlikely event,...","Drama, Comedy",6.900,44,865.0,English,United States,2015-03-01,...,4,67,22.0,"Hulu, Sky Go, Tele2 Play, Disney Plus, TV 2 Play",Contains spoilers of the first season.\r\n\r\n...,"Will Forte, Kristen Schaal, Cleopatra Coleman,...",https://image.tmdb.org/t/p/w500/9TcvdOIBEnIDXb...,No trailer available,2015,contain spoilers first season . series start r...
9582,9658,So I Married An Anti-Fan,The romance between top star Hoo Joon and his ...,"Comedy, Drama",7.420,44,50.0,Korean,"Korea, Republic of",2021-04-30,...,1,16,60.0,"Amazon Prime Video with Ads, wavve, Hulu, Raku...","_""So I married my Anti-fan""_ was a light and f...","Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",https://image.tmdb.org/t/p/w500/nfyJOt9HjdoF9q...,No trailer available,2021,_ '' marry anti-fan '' _ light fluffy show enj...
9582,9658,So I Married An Anti-Fan,The romance between top star Hoo Joon and his ...,"Comedy, Drama",7.420,44,50.0,Korean,"Korea, Republic of",2021-04-30,...,1,16,60.0,"Amazon Prime Video with Ads, wavve, Hulu, Raku...",Within less than 3 minutes going into the firs...,"Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",https://image.tmdb.org/t/p/w500/nfyJOt9HjdoF9q...,No trailer available,2021,within less 3 minutes go first episode present...
9583,9659,Porridge,Porridge is a British situation comedy broadca...,Comedy,8.127

In [13]:
sia = SentimentIntensityAnalyzer()

# Función para obtener el sentimiento del texto
def get_sentiment(text):
    scores = sia.polarity_scores(text)
    return 1 if scores["compound"]> 0.4 else (-1 if scores ["compound"]< -0.4 else 0)

In [14]:
df_reviews_separated["Sentiment"] = df_reviews_separated["Reviews"].apply(get_sentiment)
df_reviews_separated["Sentiment_Lemmatized"] = df_reviews_separated["Lemmatized_reviews"].apply(get_sentiment)

In [15]:
df_final = df_reviews_separated[["Title", "Genero", "Rating", "Reviews", "Cast", "Lemmatized_reviews", "Sentiment", "Sentiment_Lemmatized"]]

In [16]:
df_final

,Title,Genero,Rating,Reviews,Cast,Lemmatized_reviews,Sentiment,Sentiment_Lemmatized
8,Squid Game,"Action & Adventure, Mystery, Drama",7.900,Just totally worth it.,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",totally worth .,0,0
8,Squid Game,"Action & Adventure, Mystery, Drama",7.900,"Visually excellent, the dark broody streets of...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...","visually excellent , dark broody streets run n...",1,1
8,Squid Game,"Action & Adventure, Mystery, Drama",7.900,"A perfect opening and great second episode , i...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...","perfect open great second episode , let averag...",-1,0
8,Squid Game,"Action & Adventure, Mystery, Drama",7.900,extremely disappointing idea theft\r\nSo once ...,"Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...",extremely disappoint idea theft `` dangerous g...,-1,-1
8,Squid Game,"Action & Adventure, Mystery, Drama",7.900,"Whilst not an entirely original idea, ""Squid G...","Lee Jung-jae, Lee Byung-hun, Yim Si-wan, Wi Ha...","whilst entirely original idea , `` squid game ...",1,1
...,...,...,...,...,...,...,...,...
9581,The Last Man On Earth,"Drama, Comedy",6.900,Contains spoilers of the first season.\r\n\r\n...,"Will Forte, Kristen Schaal, Cleopatra Coleman,...",contain spoilers first season . series start r...,-1,-1
9582,So I Married An Anti-Fan,"Comedy, Drama",7.420,"_""So I married my Anti-fan""_ was a light and f...","Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",_ '' marry anti-fan '' _ light fluffy show enj...,1,1
9582,So I Married An Anti-Fan,"Comedy, Drama",7.420,Within less than 3 minutes going into the firs...,"Choi Tae-joon, Choi Soo-young, Hwang Chan-seon...",within less 3 minutes go first episode present...,-1,0
9583,Porridge,Comedy,8.127,It's hard to believe that there were only ever...,"Ronnie Barker, Richard Beckinsale, Fulton Mack...",'s hard believe ever twenty episodes classic b...,1,1


In [17]:
df_final[df_final["Title"]== "Yellowstone"]

,Title,Genero,Rating,Reviews,Cast,Lemmatized_reviews,Sentiment,Sentiment_Lemmatized
66,Yellowstone,"Western, Drama",8.243,Legitimately bad writing. All the scenes seem ...,"Luke Grimes, Kelly Reilly, Wes Bentley, Cole H...",legitimately bad write . scenes seem needlessl...,-1,-1
66,Yellowstone,"Western, Drama",8.243,**Cheesy and bad writing**\r\n\r\n“**Spoilers ...,"Luke Grimes, Kelly Reilly, Wes Bentley, Cole H...",* * cheesy bad write * * “ * * spoilers may in...,-1,0


In [18]:
df_final.to_csv("Dataset/df_reseñas_wordcloud.csv")